In [1]:
# Cargamos paquetes
library(dplyr)
library(abind)
library(loadeR)
library(transformeR)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: rJava

Loading required package: loadeR.java

Java version 23x amd64 by N/A detected

The maximum JVM heap space available is: 2.00 GB

NetCDF Java Library Version: 4.6.0-SNAPSHOT (Built-On: 20150423.1338) loaded and ready

You can manually set the NetCDF Java Library version before loading the package:
options(loadeR.java_forced_version = "X.Y.Z")
library(loadeR.java)

netCDF-Java CLASSPATH from bundled java package directory: /vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/lib/R/library/loadeR.java/java:/vols/abedul/home/meteo/lavinp/miniforge3/envs/C4R/lib/R/library/loadeR.java/java/netcdfAll-4.6.0-SNAPSHOT.jar

Loading required package: climate4R.UDG

climate4R.UDG version 0.2.6 (2023-06-26) is loaded

Please use 'citation("climate4R.UDG")' to cite this pa


    _______   ____  ___________________  __  ________ 
   / ___/ /  / /  |/  / __  /_  __/ __/ / / / / __  / 
  / /  / /  / / /|_/ / /_/ / / / / __/ / /_/ / /_/_/  
 / /__/ /__/ / /  / / __  / / / / /__ /___  / / \ \ 
 \___/____/_/_/  /_/_/ /_/ /_/  \___/    /_/\/   \_\ 
 
      github.com/SantanderMetGroup/climate4R



transformeR version 2.2.2 (2023-10-26) is loaded


Get the latest stable version (2.2.3) using <devtools::install_github('SantanderMetGroup/transformeR')>

Please see 'citation("transformeR")' to cite this package.



In [2]:
# Region de estudio

lon = c(-20, 15)
lat = c(27, 47)

In [3]:
# HINDCAST
anios = 1993:2016
anios = anios[anios != 1995]
meses_ini = c("03")

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {
    
    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/hindcast/psl/cmcc/35/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_hindcast_psl_cmcc_35_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "psl",
                            lonLim = lon,
                            latLim = lat,
                            season = c(04, 05, 06, 07, 08, 09)) %>% suppressMessages %>% suppressWarnings

    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
hindcast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))})

# Nombro los elementos por mes
names(hindcast) = paste0("mes_", meses_ini)

# Asigno nombre a cada grid con los leadtime
var_hcst_1 = hindcast[["mes_03"]]

# Combinamos los grids en la dimensión temporal
var_hcst_1_grid = bindGrid(var_hcst_1, dimension = "time")

# Selecciono los primeros 25 miembros
var_hcst_1_grid = subsetGrid(var_hcst_1_grid, members = 1:25)

In [4]:
# FORECAST
anios = 2021:2022
meses_ini = c("03")  # mes de inicialización para LM = 1

# Función generalizada para cargar los datos por mes y año
cargar_dato = function(anio, mes_ini) {
    
    yyyymm = paste0(anio, mes_ini)

    ruta = paste0(
        "/lustre/gmeteo/PTICLIMA/DATA/SEASONAL/",
        "seasonal-original-single-levels/medcof/forecast/psl/cmcc/35/", yyyymm, "/",
        "seasonal-original-single-levels_medcof_forecast_psl_cmcc_35_", yyyymm, ".ncml"
    )

    data_aux = loadGridData(dataset = ruta,
                            var = "psl",
                            lonLim = lon,
                            latLim = lat,
                            season = c(04, 05, 06, 07, 08, 09)) %>% suppressMessages %>% suppressWarnings

    data_aux = aggregateGrid(data_aux, aggr.d = list(FUN = "mean", na.rm = TRUE)) %>% suppressMessages %>% suppressWarnings

    return(data_aux)
}

# Creo una lista donde cada elemento es la salida para un mes de inicialización diferente
forecast = lapply(meses_ini, function(mes) {
    lapply(anios, function(anio) cargar_dato(anio, mes))})

# Nombro los elementos por mes
names(forecast) = paste0("mes_", meses_ini)

# Asigno nombre a cada grid con los leadtime
var_fcst_1 = forecast[["mes_03"]]

# Combinamos los grids en la dimensión temporal
var_fcst_1_grid = bindGrid(var_fcst_1, dimension = "time")

# Selecciono los primeros 25 miembros
var_fcst_1_members = subsetGrid(var_fcst_1_grid, members = 1:25)

# Combinamos los grids de hindcast y forecast
var_seas5_1 = bindGrid(var_hcst_1_grid, var_fcst_1_members, dimension = "time")

saveRDS(var_seas5_1, "sp_cmcc_downscaling.rds")

In [5]:
str(var_seas5_1)

List of 6
 $ Variable           :List of 2
  ..$ varName: chr "psl"
  ..$ level  : NULL
  ..- attr(*, "use_dictionary")= logi FALSE
  ..- attr(*, "description")= chr "Mean sea level pressure"
  ..- attr(*, "units")= chr "Pa"
  ..- attr(*, "longname")= chr "psl"
  ..- attr(*, "daily_agg_cellfun")= chr "none"
  ..- attr(*, "monthly_agg_cellfun")= chr "none"
  ..- attr(*, "verification_time")= chr "none"
 $ Data               : num [1:25, 1:3825, 1:21, 1:36] 101907 101883 102052 101726 102420 ...
  ..- attr(*, "dimensions")= chr [1:4] "member" "time" "lat" "lon"
 $ xyCoords           :List of 2
  ..$ x: num [1:36] -20 -19 -18 -17 -16 -15 -14 -13 -12 -11 ...
  ..$ y: num [1:21] 27 28 29 30 31 32 33 34 35 36 ...
  ..- attr(*, "projection")= chr "LatLonProjection"
  ..- attr(*, "resX")= num 1
  ..- attr(*, "resY")= num 1
 $ Dates              :List of 2
  ..$ start: chr [1:3825] "1993-04-01 01:00:00 GMT" "1993-04-02 01:00:00 GMT" "1993-04-03 01:00:00 GMT" "1993-04-04 01:00:00 GMT" ...
  ..$ 